In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import sklearn
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
import scipy
from scipy import stats

In [2]:
df=pd.read_csv("game_score_database_no_dup.csv")
df

,game_name,score,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,descriptions,lead_genre,genre_2
0,The Legend of Zelda: Ocarina of Time,99,9.1,Nintendo 64,Nintendo,76.0,Nintendo,76.0,E,11,1998,22,22,0,0,"As a young boy, Link is tricked by Ganondorf, ...",Action Adventure,Fantasy
1,Tony Hawk's Pro Skater 2,98,7.5,PlayStation,Neversoft Entertainment,80.0,Activision,68.0,T,9,2000,19,19,0,0,As most major publishers' development efforts ...,Sports,Alternative
2,Grand Theft Auto IV,98,7.8,PlayStation 3,Rockstar North,90.0,Rockstar Games,81.0,M,4,2008,64,64,0,0,[Metacritic's 2008 PS3 Game of the Year; Also ...,Action Adventure,Modern
3,SoulCalibur,98,8.5,Dreamcast,Namco,70.0,Namco,70.0,T,9,1999,24,24,0,0,"This is a tale of souls and swords, transcendi...",Action,Fighting
4,Grand Theft Auto IV,98,7.9,Xbox 360,Rockstar North,90.0,Rockstar Games,81.0,M,4,2008,86,86,0,0,[Metacritic's 2008 Xbox 360 Game of the Year; ...,Action Adventure,Modern
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19286,Vroom in the Night Sky,17,3.2,Switch,Poisoft,37.0,Poisoft,37.0,E,4,2017,15,0,0,15,Vroom in the night sky is a magical bike actio...,Sports,Individual
19287,Leisure Suit Larry: Box Office Bust,17,1.9,PlayStation 3,Team17,70.0,Funsta,21.0,M,5,2009,11,0,0,11,The Leisure Suit Larry: Box Office Bust video ...,Action Adventure,Adventure
19288,Yaris,17,4.4,Xbox 360,Backbone Entertainment,66.0,Backbone Entertainment,66.0,E10+,10,2007,7,0,0,7,[Xbox Live Arcade] Hop into a Toyota Yaris an...,Driving,Racing
19289,Ride to Hell: Retribution,16,1.4,PC,Eutechnyx,52.0,Deep Silver,67.0,M,6,2013,9,0,0,9,The game is set in the last years of the roari...,Driving,Modern


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19291 entries, 0 to 19290
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   game_name        19291 non-null  object 
 1   score            19291 non-null  int64  
 2   user_score       17933 non-null  float64
 3   platform         19291 non-null  object 
 4   developer        19291 non-null  object 
 5   developer_score  19287 non-null  float64
 6   publisher        19283 non-null  object 
 7   publisher_score  19276 non-null  float64
 8   rating           17178 non-null  object 
 9   release_month    19291 non-null  int64  
 10  release_year     19291 non-null  int64  
 11  num_crit_review  19291 non-null  int64  
 12  num_pos_critic   19291 non-null  int64  
 13  num_mix_critic   19291 non-null  int64  
 14  num_neg_critic   19291 non-null  int64  
 15  descriptions     19185 non-null  object 
 16  lead_genre       19291 non-null  object 
 17  genre_2     

In [4]:
new_df=df.copy()
new_df["user_score"].fillna(new_df.user_score.median(),inplace=True)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19291 entries, 0 to 19290
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   game_name        19291 non-null  object 
 1   score            19291 non-null  int64  
 2   user_score       19291 non-null  float64
 3   platform         19291 non-null  object 
 4   developer        19291 non-null  object 
 5   developer_score  19287 non-null  float64
 6   publisher        19283 non-null  object 
 7   publisher_score  19276 non-null  float64
 8   rating           17178 non-null  object 
 9   release_month    19291 non-null  int64  
 10  release_year     19291 non-null  int64  
 11  num_crit_review  19291 non-null  int64  
 12  num_pos_critic   19291 non-null  int64  
 13  num_mix_critic   19291 non-null  int64  
 14  num_neg_critic   19291 non-null  int64  
 15  descriptions     19185 non-null  object 
 16  lead_genre       19291 non-null  object 
 17  genre_2     

In [5]:
is_NaN = new_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = new_df[row_has_NaN]
rows_with_NaN


,game_name,score,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,descriptions,lead_genre,genre_2
68,Tetris Effect: Connected,94,8.2,Switch,Monstars Inc.,83.0,Enhance Games,87.0,NaN,10,2021,22,22,0,0,Tetris Effect: Connected is the original Tetri...,Puzzle,Stacking
130,Half-Life: Alyx,93,9.2,PC,Valve Software,85.0,Valve Software,85.0,NaN,3,2020,69,67,2,0,Half-Life: Alyx is Valve's VR return to the Ha...,Action,Shooter
132,Divinity: Original Sin II,93,8.8,PC,Larian Studios Games,83.0,Larian Studios Games,83.0,NaN,9,2017,72,71,1,0,"Master deep, tactical combat. Join up to 3 oth...",Action RPG,Role-Playing
166,Beat Saber,93,8.5,PC,Hyperbolic Magnetism,90.0,Hyperbolic Magnetism,90.0,NaN,5,2019,7,7,0,0,Beat Saber is a VR rhythm game where your goal...,Action,Rhythm
180,Wipeout XL,93,8.7,PlayStation,Psygnosis,89.0,SCEE,72.0,NaN,9,1996,8,8,0,0,The original scorched the game world and becam...,Driving,Racing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19239,Fray,27,2.4,PC,Brain Candy,27.0,Brain Candy,27.0,NaN,2,2012,8,0,2,6,"Fray is a gritty sci-fi game set in 2098, a wo...",Strategy,Real-Time
19261,Stalin vs. Martians,25,4.4,PC,N-Game Studios,41.0,Mezmer Games,25.0,NaN,4,2009,16,0,2,14,"Stalin vs. Martians, as the name suggests, is ...",Strategy,Real-Time
19267,Postal III,24,3.1,PC,Running With Scissors,47.0,Running With Scissors,47.0,NaN,12,2011,20,0,1,19,The Postal series continues in this third inst...,First-Person,Modern
19272,FlatOut 3: Chaos & Destruction,23,3.0,PC,Team6 Game Studios,41.0,Strategy First,62.0,NaN,12,2011,22,0,2,20,FlatOut 3: Chaos & Destruction brings a new di...,Driving,Racing


In [6]:
df[df["lead_genre"]=='Sim']

,game_name,score,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,descriptions,lead_genre,genre_2
795,FIFA 14,88,5.8,Xbox One,EA Canada,75.0,EA Sports,78.0,E,11,2013,22,22,0,0,Shooting has been transformed. Players now hav...,Sim,Sports
1321,NBA 2K14,86,5.7,Xbox One,Visual Concepts,79.0,2K Sports,72.0,E,11,2013,15,15,0,0,"NBA 2K14 promises to raise the bar yet again, ...",Sim,Sports
7556,Madden NFL 13,75,7.0,Wii U,EA Tiburon,76.0,EA Sports,78.0,E,11,2012,9,6,3,0,Build your own legacy as a player or a coach—o...,Sim,Sports
11823,FIFA Soccer 13,69,6.5,Wii U,Electronic Arts,73.0,EA Sports,78.0,E,11,2012,28,13,13,2,Innovations that make use of the Wii U GamePad...,Sim,Sports


highly correlated columns might impact negatively on the model, 


In [7]:
new_df["rating"].fillna(new_df.rating.mode()[0],inplace=True)

In [8]:
new_df.dropna(inplace=True)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19164 entries, 0 to 19290
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   game_name        19164 non-null  object 
 1   score            19164 non-null  int64  
 2   user_score       19164 non-null  float64
 3   platform         19164 non-null  object 
 4   developer        19164 non-null  object 
 5   developer_score  19164 non-null  float64
 6   publisher        19164 non-null  object 
 7   publisher_score  19164 non-null  float64
 8   rating           19164 non-null  object 
 9   release_month    19164 non-null  int64  
 10  release_year     19164 non-null  int64  
 11  num_crit_review  19164 non-null  int64  
 12  num_pos_critic   19164 non-null  int64  
 13  num_mix_critic   19164 non-null  int64  
 14  num_neg_critic   19164 non-null  int64  
 15  descriptions     19164 non-null  object 
 16  lead_genre       19164 non-null  object 
 17  genre_2     

In [9]:
new_df.lead_genre.unique()

array(['Action Adventure', 'Sports', 'Action', 'Modern', 'Role-Playing',
       'Fantasy', 'Adventure', 'Driving', 'Miscellaneous', 'Puzzle',
       'Strategy', 'Action RPG', 'General', 'Platformer', 'Racing',
       'Simulation', 'Other', 'Shooter', 'Compilation', 'PC-style RPG',
       'First-Person', 'Horror', 'Sim', 'Sci-Fi', 'Card Battle',
       'Japanese-Style', 'Massively Multiplayer Online', 'Ice Hockey',
       'Dancing', 'City Building', 'Street', 'Console-style RPG',
       'Fighting', 'Traditional', '2D', 'Pinball', 'Arcade', 'Historic',
       'Third-Person', 'Scrolling', 'Open-World', 'Real-Time',
       'GT / Street', 'Music', 'Tycoon', 'Party / Minigame',
       'Breeding/Constructing', 'Alternative', 'Rally / Offroad',
       'Military', 'WWI', 'Virtual Life', 'Edutainment', 'MOBA', '3D',
       'Party', '"Beat-\'Em-Up"', 'Olympic Sports',
       'Massively Multiplayer', 'Government', 'Formula One', 'Futuristic',
       'Flight', 'WWII'], dtype=object)

In [10]:
new_df[new_df["lead_genre"]=='First-Person']

,game_name,score,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,descriptions,lead_genre,genre_2
710,L.A. Noire,89,7.9,Xbox 360,Team Bondi,77.0,Rockstar Games,81.0,M,5,2011,83,78,5,0,Rockstar Games and Team Bondi are crafting a f...,First-Person,Action Adventure
1035,Steins;Gate,87,9.0,PC,M2,77.0,Jast,87.0,M,3,2014,9,9,0,0,"Set in Akihabara, the Mecca of Japanese anime ...",First-Person,Modern
2250,Tom Clancy's The Division 2,84,4.6,PC,Massive Entertainment,79.0,Ubisoft,69.0,M,3,2019,34,32,2,0,"Set in Washington D.C., the game takes place 7...",First-Person,Action
3002,Tom Clancy's The Division 2,82,6.2,PlayStation 4,Massive Entertainment,79.0,Ubisoft,69.0,M,3,2019,56,49,6,1,"Set in Washington D.C., the game takes place 7...",First-Person,Action
3051,Tom Clancy's The Division 2,82,5.9,Xbox One,Massive Entertainment,79.0,Ubisoft,69.0,M,3,2019,30,26,3,1,"Set in Washington D.C., the game takes place 7...",First-Person,Action
4513,World of Tanks,80,3.4,PC,Wargaming.net,75.0,Wargaming.net,75.0,T,4,2011,38,29,9,0,World of Tanks is the first and only team-base...,First-Person,Modern
6326,The Elder Scrolls Online: Tamriel Unlimited,77,8.3,Xbox One,ZeniMax Media,76.0,Bethesda Softworks,74.0,M,6,2015,11,8,3,0,The Elder Scrolls Online: Tamriel Unlimited in...,First-Person,Massively Multiplayer Online
8830,The Elder Scrolls Online: Tamriel Unlimited,74,7.6,PlayStation 4,ZeniMax Media,76.0,Bethesda Softworks,74.0,M,6,2015,30,17,13,0,The Elder Scrolls Online: Tamriel Unlimited in...,First-Person,Massively Multiplayer Online
10497,The Elder Scrolls Online,71,5.7,PC,ZeniMax Media,76.0,Bethesda Softworks,74.0,M,4,2014,64,27,36,1,Experience this epic adventure on your own or ...,First-Person,Role-Playing
19267,Postal III,24,3.1,PC,Running With Scissors,47.0,Running With Scissors,47.0,T,12,2011,20,0,1,19,The Postal series continues in this third inst...,First-Person,Modern


In [11]:
new_df.genre_2.unique()

array(['Fantasy', 'Alternative', 'Modern', 'Fighting', 'Platformer',
       'Open-World', 'Action Adventure', 'General', 'Shooter',
       'Traditional', 'Horror', 'Miscellaneous', 'Action RPG',
       'Visual Novel', 'First-Person', 'Racing', 'Compilation',
       'PC-style RPG', 'Adventure', 'Japanese-Style', 'Stacking',
       'Puzzle', 'Console-style RPG', 'Linear', 'Turn-Based', 'Survival',
       'Third-Person', 'Role-Playing', 'Real-Time', 'Western-Style', '3D',
       'Rhythm', '"Beat-\'Em-Up"', 'Simulation', 'Sci-Fi',
       'Massively Multiplayer', 'Team', 'Flight', 'Action', 'Historic',
       'Car Combat', 'Virtual', 'Management', '2D', 'Scrolling', 'Text',
       'Submarine', 'Kart', 'Individual', 'Futuristic', 'Tycoon',
       'Sports', 'Small Spaceship', 'Point-and-Click', 'Strategy',
       'Massively Multiplayer Online', 'Trainer', 'Party', 'GT / Street',
       'Virtual Life', 'Mission-based', 'Card Battle', 'Tank', 'Logic',
       'Sandbox', 'Pinball', 'Sim', 'Drivin

In [12]:
temp=new_df[new_df["genre_2"]=='simulator Life']
temp

,game_name,score,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,descriptions,lead_genre,genre_2


In [13]:
#using raplace method with regex option to lower the dimension of the genre column, every genre is replaced with a corresponding bigger genre
new_df['genre_2']=new_df['genre_2'].str.replace(r'(Fantasy|General|Visual Novel|Adventure|Third-Person|3D|Sci-Fi|2D|Text|Point-and-Click|Trainer|Sandbox|Hidden Object|Videos|Platformer)', 'adventure', regex=True)
new_df['genre_2']=new_df['genre_2'].str.replace(r'(Action RPG|First-Person|PC-style RPG|Japanese-Style|Console-style RPG|Role-Playing|Western-Style|Marine|Massively Multiplayer|Massively Multiplayer Online|MOBA|RPG Online)', 'RPG', regex=True)
new_df['genre_2']=new_df['genre_2'].str.replace(r'(Turn-Based|Real-Time|Wargame|Business / Tycoon|Tactical)', 'Strategy', regex=True)
new_df['genre_2']=new_df['genre_2'].str.replace(r'(Alternative|Traditional|Team|Car Combat|Individual|Sports|Exercise / Fitness|Football|Nature|Soccer|Olympic Sports|Other|Gambling|Biking|Volleyball|Racing|Simulation|GT / Street|Driving|Demolition Derby|Rally / Offroad)', 'sports', regex=True)
new_df['genre_2']=new_df['genre_2'].str.replace(r'(Flight|Virtual|Management|Tycoon|Small Spaceship|Virtual Life|Breeding/Constructing|City Building|Train|Application)', 'simulator', regex=True)
new_df['genre_2']=new_df['genre_2'].str.replace(r'(Modern|Open-World|Action adventure|Shooter|Horror|Compilation|Linear|Survival|"Beat-\'Em-Up"|Action|Historic|Scrolling|Submarine|Mission-based|Tank|Roguelike|Large Spaceship|Space|Vehicle|Ship|Rail|Fighting)', 'action', regex=True)
new_df['genre_2']=new_df['genre_2'].str.replace(r'(Stacking|Puzzle|Rhythm|Kart|Futuristic|Card Battle|Logic|Pinball|Arcade|Matching|Board / Card Game|Board Games|Music Maker|Trivia / Game Show|Edutainment|Party / Minigame|Parlor|Music|"Shoot-\'Em-Up"|Miscellaneous|Party)', 'arcade', regex=True)






In [14]:
#dealing with special cases
new_df.replace('Sim','sports',inplace=True)
new_df.replace('simulator Life','simulator',inplace=True)
new_df.replace('RPG Online', 'RPG',inplace=True)

In [15]:
new_df.genre_2.value_counts()

action       5590
adventure    5572
sports       2814
arcade       1778
RPG          1679
Strategy     1305
simulator     426
Name: genre_2, dtype: int64

In [16]:
new_df.genre_2.unique()

array(['adventure', 'sports', 'action', 'arcade', 'RPG', 'Strategy',
       'simulator'], dtype=object)

In [17]:
new_df['release_month'] = pd.Categorical(new_df['release_month'])

In [27]:
new_df=new_df[["game_name","user_score","platform","developer","developer_score","publisher","publisher_score","rating","release_month","release_year","num_crit_review","num_pos_critic","num_mix_critic","num_neg_critic","descriptions","lead_genre","genre_2","score"]]
new_df.to_csv("dataframe_for_visual.csv",index=False)
new_df

,game_name,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,descriptions,lead_genre,genre_2,score
0,The Legend of Zelda: Ocarina of Time,9.1,Nintendo 64,Nintendo,76.0,Nintendo,76.0,E,11,1998,22,22,0,0,"As a young boy, Link is tricked by Ganondorf, ...",Action Adventure,adventure,99
1,Tony Hawk's Pro Skater 2,7.5,PlayStation,Neversoft Entertainment,80.0,Activision,68.0,T,9,2000,19,19,0,0,As most major publishers' development efforts ...,Sports,sports,98
2,Grand Theft Auto IV,7.8,PlayStation 3,Rockstar North,90.0,Rockstar Games,81.0,M,4,2008,64,64,0,0,[Metacritic's 2008 PS3 Game of the Year; Also ...,Action Adventure,action,98
3,SoulCalibur,8.5,Dreamcast,Namco,70.0,Namco,70.0,T,9,1999,24,24,0,0,"This is a tale of souls and swords, transcendi...",Action,action,98
4,Grand Theft Auto IV,7.9,Xbox 360,Rockstar North,90.0,Rockstar Games,81.0,M,4,2008,86,86,0,0,[Metacritic's 2008 Xbox 360 Game of the Year; ...,Action Adventure,action,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19286,Vroom in the Night Sky,3.2,Switch,Poisoft,37.0,Poisoft,37.0,E,4,2017,15,0,0,15,Vroom in the night sky is a magical bike actio...,Sports,sports,17
19287,Leisure Suit Larry: Box Office Bust,1.9,PlayStation 3,Team17,70.0,Funsta,21.0,M,5,2009,11,0,0,11,The Leisure Suit Larry: Box Office Bust video ...,Action Adventure,adventure,17
19288,Yaris,4.4,Xbox 360,Backbone Entertainment,66.0,Backbone Entertainment,66.0,E10+,10,2007,7,0,0,7,[Xbox Live Arcade] Hop into a Toyota Yaris an...,Driving,sports,17
19289,Ride to Hell: Retribution,1.4,PC,Eutechnyx,52.0,Deep Silver,67.0,M,6,2013,9,0,0,9,The game is set in the last years of the roari...,Driving,action,16


In [19]:
df_for_model=new_df.drop(['lead_genre','num_crit_review','publisher','developer','descriptions','game_name'],axis=1)
df_for_model

,user_score,platform,developer_score,publisher_score,rating,release_month,release_year,num_pos_critic,num_mix_critic,num_neg_critic,genre_2,score
0,9.1,Nintendo 64,76.0,76.0,E,11,1998,22,0,0,adventure,99
1,7.5,PlayStation,80.0,68.0,T,9,2000,19,0,0,sports,98
2,7.8,PlayStation 3,90.0,81.0,M,4,2008,64,0,0,action,98
3,8.5,Dreamcast,70.0,70.0,T,9,1999,24,0,0,action,98
4,7.9,Xbox 360,90.0,81.0,M,4,2008,86,0,0,action,98
...,...,...,...,...,...,...,...,...,...,...,...,...
19286,3.2,Switch,37.0,37.0,E,4,2017,0,0,15,sports,17
19287,1.9,PlayStation 3,70.0,21.0,M,5,2009,0,0,11,adventure,17
19288,4.4,Xbox 360,66.0,66.0,E10+,10,2007,0,0,7,sports,17
19289,1.4,PC,52.0,67.0,M,6,2013,0,0,9,action,16


In [20]:
catagory_list=['platform','rating','genre_2']
for catag in catagory_list:
        df_for_model[catag]= df_for_model[catag].astype('category')
        df_for_model=pd.get_dummies(df_for_model,columns=[catag],prefix=[catag])

In [21]:
df_for_model.columns

Index(['user_score', 'developer_score', 'publisher_score', 'release_month',
       'release_year', 'num_pos_critic', 'num_mix_critic', 'num_neg_critic',
       'score', 'platform_3DS', 'platform_DS', 'platform_Dreamcast',
       'platform_Game Boy Advance', 'platform_GameCube',
       'platform_Nintendo 64', 'platform_PC', 'platform_PSP',
       'platform_PlayStation', 'platform_PlayStation 2',
       'platform_PlayStation 3', 'platform_PlayStation 4',
       'platform_PlayStation 5', 'platform_PlayStation Vita',
       'platform_Stadia', 'platform_Switch', 'platform_Wii', 'platform_Wii U',
       'platform_Xbox', 'platform_Xbox 360', 'platform_Xbox One',
       'platform_Xbox Series X', 'rating_AO', 'rating_E', 'rating_E10+',
       'rating_K-A', 'rating_M', 'rating_RP', 'rating_T', 'genre_2_RPG',
       'genre_2_Strategy', 'genre_2_action', 'genre_2_adventure',
       'genre_2_arcade', 'genre_2_simulator', 'genre_2_sports'],
      dtype='object')

In [22]:
df_for_model=df_for_model[["user_score","developer_score","publisher_score","release_month","release_year",
                           "num_pos_critic","num_mix_critic","num_neg_critic","platform_3DS","platform_DS",
                           "platform_Dreamcast","platform_Game Boy Advance","platform_GameCube","platform_Nintendo 64",
                           "platform_PC","platform_PSP","platform_PlayStation","platform_PlayStation 2","platform_PlayStation 3",
                           "platform_PlayStation 4","platform_PlayStation 5","platform_PlayStation Vita","platform_Stadia",
                           "platform_Switch","platform_Wii","platform_Wii U","platform_Xbox","platform_Xbox 360","platform_Xbox One",
                           "platform_Xbox Series X","rating_AO","rating_E","rating_E10+","rating_K-A","rating_M","rating_RP",
                          "rating_T","genre_2_RPG","genre_2_Strategy","genre_2_action","genre_2_adventure","genre_2_arcade",
                           "genre_2_simulator","genre_2_sports","score"]]
df_for_model

,user_score,developer_score,publisher_score,release_month,release_year,num_pos_critic,num_mix_critic,num_neg_critic,platform_3DS,platform_DS,...,rating_RP,rating_T,genre_2_RPG,genre_2_Strategy,genre_2_action,genre_2_adventure,genre_2_arcade,genre_2_simulator,genre_2_sports,score
0,9.1,76.0,76.0,11,1998,22,0,0,0,0,...,0,0,0,0,0,1,0,0,0,99
1,7.5,80.0,68.0,9,2000,19,0,0,0,0,...,0,1,0,0,0,0,0,0,1,98
2,7.8,90.0,81.0,4,2008,64,0,0,0,0,...,0,0,0,0,1,0,0,0,0,98
3,8.5,70.0,70.0,9,1999,24,0,0,0,0,...,0,1,0,0,1,0,0,0,0,98
4,7.9,90.0,81.0,4,2008,86,0,0,0,0,...,0,0,0,0,1,0,0,0,0,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19286,3.2,37.0,37.0,4,2017,0,0,15,0,0,...,0,0,0,0,0,0,0,0,1,17
19287,1.9,70.0,21.0,5,2009,0,0,11,0,0,...,0,0,0,0,0,1,0,0,0,17
19288,4.4,66.0,66.0,10,2007,0,0,7,0,0,...,0,0,0,0,0,0,0,0,1,17
19289,1.4,52.0,67.0,6,2013,0,0,9,0,0,...,0,0,0,0,1,0,0,0,0,16


In [23]:
#this code was given to us from a friend that works as a data scientist
scaled_list=['user_score','developer_score','publisher_score','num_pos_critic','num_neg_critic','num_mix_critic']
for cat in scaled_list:
    
    k2, p = stats.normaltest(df_for_model[cat])
    alpha = 0.05

    if p < alpha:  # null hypothesis: x comes from a normal distribution
        vec = np.array(df_for_model[cat])
        pt = PowerTransformer(method='yeo-johnson')
        vec = vec.reshape(-1, 1)
        pt.fit(vec)
        transformed_data = pt.transform(vec)
        transformed_data = [i[0] for i in transformed_data]
        df_for_model[cat] = transformed_data       

    else:
        vec = np.array(df_for_model[cat])
        scale = StandardScaler()
        vec = vec.reshape(-1, 1)
        scale.fit(vec)
        transformed_data = scale.transform(vec)
        transformed_data = [i[0] for i in transformed_data]
        df_for_model[cat] = transformed_data


The null hypothesis can be rejected - Not Gaussian
USE: NORMALIZATION
The null hypothesis can be rejected - Not Gaussian
USE: NORMALIZATION
The null hypothesis can be rejected - Not Gaussian
USE: NORMALIZATION
The null hypothesis can be rejected - Not Gaussian
USE: NORMALIZATION
The null hypothesis can be rejected - Not Gaussian
USE: NORMALIZATION
The null hypothesis can be rejected - Not Gaussian
USE: NORMALIZATION


In [24]:
df_for_model

,user_score,developer_score,publisher_score,release_month,release_year,num_pos_critic,num_mix_critic,num_neg_critic,platform_3DS,platform_DS,...,rating_RP,rating_T,genre_2_RPG,genre_2_Strategy,genre_2_action,genre_2_adventure,genre_2_arcade,genre_2_simulator,genre_2_sports,score
0,2.297656,0.672474,0.991054,11,1998,0.938100,-2.067688,-0.822358,0,0,...,0,0,0,0,0,1,0,0,0,99
1,0.278088,1.308206,-0.404405,9,2000,0.805895,-2.067688,-0.822358,0,0,...,0,1,0,0,0,0,0,0,1,98
2,0.598264,3.192685,1.993658,4,2008,1.952246,-2.067688,-0.822358,0,0,...,0,0,0,0,1,0,0,0,0,98
3,1.447404,-0.164749,-0.078875,9,1999,1.017443,-2.067688,-0.822358,0,0,...,0,1,0,0,1,0,0,0,0,98
4,0.710655,3.192685,1.993658,4,2008,2.247038,-2.067688,-0.822358,0,0,...,0,0,0,0,1,0,0,0,0,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19286,-2.122507,-2.740225,-3.700344,4,2017,-1.803902,-2.067688,1.776511,0,0,...,0,0,0,0,0,0,0,0,1,17
19287,-2.309357,-0.164749,-4.358330,5,2009,-1.803902,-2.067688,1.732520,0,0,...,0,0,0,0,0,1,0,0,0,17
19288,-1.783429,-0.650037,-0.714838,10,2007,-1.803902,-2.067688,1.639806,0,0,...,0,0,0,0,0,0,0,0,1,17
19289,-2.345944,-1.940564,-0.561491,6,2013,-1.803902,-2.067688,1.696105,0,0,...,0,0,0,0,1,0,0,0,0,16


In [25]:
df_for_model.to_csv("dataframe_for_model.csv",index=False)